<a href="https://colab.research.google.com/github/arjunarora1201/Audio-To-Text-/blob/main/SpeechRecognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libraries

In [17]:
pip install SpeechRecognition pydub

In [16]:
!sudo apt update && sudo apt install -y ffmpeg

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [4,710 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,747 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 P

In [18]:
import speech_recognition as sr
import os
import subprocess
from google.colab import files
from pydub import AudioSegment
from pydub.silence import split_on_silence

Upload Audio File

In [20]:
uploaded = files.upload()
input_filename = next(iter(uploaded))

Saving 01.mp3 to 01 (1).mp3


Convert Mp3 to Wav format

In [22]:
try:
    # First try with pydub
    sound = AudioSegment.from_file(input_filename)
    wav_filename = "converted_audio.wav"
    sound.export(wav_filename, format="wav")
except:
    # Fallback to ffmpeg if pydub fails
    !ffmpeg -i "{input_filename}" -acodec pcm_s16le -ar 16000 "{wav_filename}" -y

if not os.path.exists(wav_filename):
    raise FileNotFoundError(f"Conversion failed: {wav_filename} not created")

Main Function

In [23]:
r = sr.Recognizer()

def get_large_audio_transcription(path):
    """Split audio into chunks and recognize speech"""
    try:
        sound = AudioSegment.from_wav(path)
    except:
        # If standard WAV load fails, try forcing format
        sound = AudioSegment.from_file(path, format="wav")

    chunks = split_on_silence(
        sound,
        min_silence_len=500,
        silence_thresh=sound.dBFS-14,
        keep_silence=500
    )

    folder_name = "audio-chunks"
    os.makedirs(folder_name, exist_ok=True)

    whole_text = ""

    for i, chunk in enumerate(chunks, start=1):
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        chunk.export(chunk_filename, format="wav")

        with sr.AudioFile(chunk_filename) as source:
            audio = r.record(source)
            try:
                text = r.recognize_google(audio)
                text = f"{text.capitalize()}. "
                print(f"Chunk {i}: {text}")
                whole_text += text
            except sr.UnknownValueError:
                print(f"Chunk {i}: Could not understand audio")
            except sr.RequestError as e:
                print(f"Chunk {i}: API error; {e}")

    return whole_text

In [24]:
print("\nStarting transcription...")
result = get_large_audio_transcription(wav_filename)
print("\nFull transcription:")
print(result)


Starting transcription...
Chunk 1: Day 8. 
Chunk 2: Could not understand audio
Chunk 3: Describing a person. 
Chunk 4: Q card. 
Chunk 5: Describe a famous person you would like to meet. 
Chunk 6: You should say who it is. 
Chunk 7: What they are famous for. 
Chunk 8: What you would do if you meet them. 
Chunk 9: And explain why you would like to meet them. 
Chunk 10: Sample answer. 
Chunk 11: Well the person i'd like to meet most is johnny mo. 
Chunk 12: Who is a very famous film director in china. 
Chunk 13: I've always admired him. 
Chunk 14: He's a man of average height in his forties i think. 
Chunk 15: So far he has directed many good films. 
Chunk 16: He's well-known in china for his versatile talents. 
Chunk 17: First of all he's a famous director. 
Chunk 18: Most of his films have won international prizes. 
Chunk 19: And the actresses who have appeared in his films would consequently become popular as well. 
Chunk 20: I like his film the road home. 
Chunk 21: Which describes h